In [1]:
# Loadlibraries
import os
import re
from openai import OpenAI
from pydantic import BaseModel
import pandas as pd
from pandas import json_normalize
import numpy as np
import json
# Set model to use GPU
import torch
torch.cuda.is_available()

True

In [2]:
## Reading in the csv file, from ENGLISH language Rules-Based NLP results, of the extracted names and coordinates of
## the towns, villages, and named features of the Plateau, creating a pandas dataframe with these locations and their
## coordinates, then creating a list of just the location names, which then becomes one of the lists of keywords.

df_PlatLoc = pd.read_csv("YOUR_DATA_keywords")
df_PlatLoc = df_PlatLoc.dropna()
df_PlatLoc = df_PlatLoc[["Plateau_Locations", "LAT", "LON", "ALT"]]
df_PlatLoc["LAT"] = pd.to_numeric(df_PlatLoc["LAT"], downcast="float")
df_PlatLoc["LON"] = pd.to_numeric(df_PlatLoc["LON"], downcast="float")
df_PlatLoc["ALT"] = pd.to_numeric(df_PlatLoc["ALT"], downcast="float")
df_PlatLoc.columns = ["Plateau_Locations", "LAT", "LON", "ALT"]
print(df_PlatLoc)
PlatLoc = df_PlatLoc["Plateau_Locations"].to_list()
print(PlatLoc)

        Plateau_Locations        LAT       LON     ALT
0      Chambon-sur-Lignon  45.060810  4.302941   961.0
1                     Fay  44.985435  4.226421  1181.0
2          Fay-sur-Lignon  44.985435  4.226421  1181.0
3              La Fayolle  45.053501  4.285560  1002.0
4              Le Chambon  45.060810  4.302941   961.0
5   Le Chambon sur Lignon  45.060810  4.302941   961.0
6   Le Chambon-sur-Lignon  45.060810  4.302941   961.0
7                Le Mazet  45.047192  4.244734  1027.0
8               Les Tavas  45.051365  4.338720  1005.0
9                   Mazet  45.047192  4.244734  1027.0
10                Plateau  45.060810  4.302941   961.0
11           Saint-Agreve  45.010014  4.395049  1066.0
12           Saint-Jeures  45.095669  4.205091  1043.0
13                  Tence  45.116135  4.289156   849.0
['Chambon-sur-Lignon', 'Fay', 'Fay-sur-Lignon', 'La Fayolle', 'Le Chambon', 'Le Chambon sur Lignon', 'Le Chambon-sur-Lignon', 'Le Mazet', 'Les Tavas', 'Mazet', 'Plateau', 'Sai

In [3]:
## Reading in the csv file, from ENGLISH language Rules-Based NLP results, of the extracted names and coordinates of
## the other locations in France (not on Plateau), creating a pandas dataframe with these locations and their
## coordinates, then creating a list of just the location names, which then becomes one of the lists of keywords.

df_FranceLoc = pd.read_csv("YOUR_DATA_keywords")
df_FranceLoc = df_FranceLoc.dropna()
df_FranceLoc = df_FranceLoc[["other_French_Locations", "LAT", "LON", "ALT"]]
df_FranceLoc["LAT"] = pd.to_numeric(df_FranceLoc["LAT"], downcast="float")
df_FranceLoc["LON"] = pd.to_numeric(df_FranceLoc["LON"], downcast="float")
df_FranceLoc["ALT"] = pd.to_numeric(df_FranceLoc["ALT"], downcast="float")
df_FranceLoc.columns = ["other_French_Locations", "LAT", "LON", "ALT"]
print(df_FranceLoc)
FranceLoc = df_FranceLoc["other_French_Locations"].to_list()
print(FranceLoc)

   other_French_Locations        LAT       LON    ALT
0               Annemasse  46.194389  6.237750  441.0
1            Camp de Gurs  43.273762 -0.740310  186.0
2      Camp Saint-Cyprien  42.636631  3.034213    2.0
3                  Drancy  48.925831  2.445280   47.0
4                    Gurs  43.273762 -0.740310  186.0
5                  Le Puy  45.043659  3.885230  640.0
6         Le Puy-en-Velay  45.043659  3.885230  640.0
7                    Lyon  45.748459  4.846710  173.0
8               Marseille  43.296951  5.381070   28.0
9              Marseilles  43.296951  5.381070   28.0
10                  Paris  48.853409  2.348800   42.0
11                    Puy  45.043659  3.885230  640.0
12             Rivesaltes  42.807911  2.890970   30.0
13          Saint Étienne  45.433891  4.390000  529.0
14          Saint-Cyprien  42.636631  3.034213    2.0
15          Saint-Etienne  45.433891  4.390000  529.0
16          Saint-Étienne  45.433891  4.390000  529.0
17            St. Etienne  4

In [4]:
## Reading in the csv file, from ENGLISH language Rules-Based NLP results, of the extracted names and coordinates of
## the Childrens Homes on the Plateau, creating a pandas dataframe with these locations and their
## coordinates, then creating a list of just the location names, which then becomes one of the lists of keywords.

df_homes = pd.read_csv("YOUR_DATA_keywords")
df_homes = df_homes.dropna()
df_homes = df_homes[["Childrens_Homes", "LAT", "LON", "ALT"]]
df_homes["LAT"] = pd.to_numeric(df_homes["LAT"], downcast="float")
df_homes["LON"] = pd.to_numeric(df_homes["LON"], downcast="float")
df_homes["ALT"] = pd.to_numeric(df_homes["ALT"], downcast="float")
df_homes.columns = ["Childrens_Homes", "LAT", "LON", "ALT"]
print(df_homes)
homes = df_homes["Childrens_Homes"].to_list()
print(homes)

         Childrens_Homes        LAT       LON     ALT
0          Côteau Fleuri  45.052872  4.340592  1028.0
1               crickets  45.071281  4.326289  1067.0
2                Faïdoli  45.055611  4.289219   976.0
3               Grillons  45.071281  4.326289  1067.0
4                 Guespy  45.062374  4.312919  1006.0
5     House of the Rocks  45.052620  4.321041   959.0
6              La Guespy  45.062374  4.312919  1006.0
7                L'Abric  45.060558  4.311144   997.0
8          L'Abric House  45.060558  4.311144   997.0
9             Le Faïdoli  45.055611  4.289219   976.0
10           Les Grillon  45.071281  4.326289  1067.0
11          Les Grillons  45.071281  4.326289  1067.0
12            Les Roches  45.052620  4.321041   959.0
13  Maison des de Roches  45.052620  4.321041   959.0
14     Maison des Roches  45.052620  4.321041   959.0
15    Pension De Famille  45.071281  4.326289  1067.0
16               Rohnert  45.052620  4.321041   959.0
17          the crickets  45

In [5]:
## Reading in the csv file, from ENGLISH language Rules-Based NLP results, of the extracted names and coordinates of
## the Farms on the Plateau, creating a pandas dataframe with these locations and their
## coordinates, then creating a list of just the location names, which then becomes one of the lists of keywords.

df_farms = pd.read_csv("YOUR_DATA_keywords")
df_farms = df_farms.dropna()
df_farms = df_farms[["Farms", "LAT", "LON", "ALT"]]
df_farms["LAT"] = pd.to_numeric(df_farms["LAT"], downcast="float")
df_farms["LON"] = pd.to_numeric(df_farms["LON"], downcast="float")
df_farms["ALT"] = pd.to_numeric(df_farms["ALT"], downcast="float")
df_farms.columns = ["Farms", "LAT", "LON", "ALT"]
print(df_farms)
farms = df_farms["Farms"].to_list()
print(farms)

         Farms        LAT       LON    ALT
0  La Bergerie  45.131886  4.284475  822.0
['La Bergerie']


In [6]:
## Reading in the csv file, from ENGLISH language Rules-Based NLP results, of the extracted names and coordinates of
## the Schools on the Plateau, creating a pandas dataframe with these locations and their
## coordinates, then creating a list of just the location names, which then becomes one of the lists of keywords.

df_schools = pd.read_csv("YOUR_DATA_keywords")
df_schools = df_schools.dropna()
df_schools = df_schools[["Schools", "LAT", "LON", "ALT"]]
df_schools["LAT"] = pd.to_numeric(df_schools["LAT"], downcast="float")
df_schools["LON"] = pd.to_numeric(df_schools["LON"], downcast="float")
df_schools["ALT"] = pd.to_numeric(df_schools["ALT"], downcast="float")
df_schools.columns = ["Schools", "LAT", "LON", "ALT"]
print(df_schools)
schools = df_schools["Schools"].to_list()
print(schools)

           Schools        LAT       LON     ALT
0          Cévenol  45.060959  4.303929   961.0
1  College Cevenol  45.060959  4.303929   961.0
2  College Cévenol  45.060959  4.303929   961.0
3  Collège Cévenol  45.060959  4.303929   961.0
4      Ferme École  45.055912  4.325135  1005.0
5   Studio Cévenol  45.058929  4.301951   942.0
['Cévenol', 'College Cevenol', 'College Cévenol', 'Collège Cévenol', 'Ferme École', 'Studio Cévenol']


In [7]:
## Reading in the csv file, from ENGLISH language Rules-Based NLP results, of the extracted non-named
## features mentioned in the testimonies, creating a pandas dataframe with features,
## then creating a list of the features, which then becomes one of the lists of keywords.

df_features = pd.read_csv("YOUR_DATA_keywords")
df_features = df_features.dropna()
df_features = df_features[["Features"]]
df_features.columns = ["Features"]
print(df_features)
features = df_features["Features"].to_list()
print(features)

      Features
0      village
1        house
2       school
3         home
4         farm
..         ...
74        bank
75        mill
76  university
77       wagon
78       fence

[79 rows x 1 columns]
['village', 'house', 'school', 'home', 'farm', 'train', 'camp', 'church', 'woods', 'bus', 'homes', 'farms', 'mountains', 'border', 'station', 'garden', 'camps', 'houses', 'mountain', 'plateau', 'apartment', 'pass', 'concentration camps', 'railroad', 'car', 'road', 'college', 'barn', 'tree', 'valley', 'fields', 'store', 'field', 'forest', 'hotel', 'borders', 'trains', 'wall', 'hill', 'concentration camp', 'bridge', 'office', 'forests', 'trucks', 'universities', 'roads', 'schools', 'museum', 'railway station', 'churches', 'walls', 'truck', 'corridor', 'police station', 'airport', 'orchards', 'wagons', 'hospital', 'desert', 'peak', 'factory', 'monasteries', 'estate', 'stables', 'barracks', 'chateau', 'monastery', 'facility', 'museums', 'gardens', 'gate', 'trees', 'prison', 'banks', 'bank',

In [3]:
# Bring in the sample dataset, the smaller sub-corpus.
path = "YOUR_DATA_test"

def read_txt_files(directory):
    # Reads all .txt files in a directory and returns a combined string of their contents.

    file_contents = ''
    
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            filepath = os.path.join(directory, filename)
            with open(filepath, "r", encoding="utf8") as f:
                file_contents = file_contents + (f.read())
    return file_contents

texts = read_txt_files(path)

In [4]:
# Developing function to identify and resolve toponyms, and detect emotions in 150 word context 
# on either side of each toponym.

# Set a global variable for my OpenAI API key and access model
os.environ["OPENAI_API_KEY"] = "YOUR_KEY"
client = OpenAI()

# Bring "Instructions" txt file into memory for the response function to access

with open("openai_EmotionSpatialDetect_prompt_23.txt", "r", encoding="utf-8") as f:
    instructions = f.read()

# Alternative data for testing:
#texts = "I know them by sight. We're becoming friends now. I'm glad not to have to travel alone. Age 16, November 1941, Le Chambon-sur-Lignon, France. I'm in Le Chambon-sur-Lignon. It's way up on a high plateau in the mountains. It's beautiful here. What an amazing difference from Gurs! I'm staying at La Guespy, which is a home for refugees.  My friend just arrived, and before she came she saw her mother for an hour that morning through the fence, and then they put them all on cattle wagons and they were deported. And now they're gone.  We are not sure, but we think they took them to Drancy.  Martin and I are working in the new carpentry shop now. I don't really like carpentry, but at least I know how to do it, and at least I am safe."

# Setting my variable of keywords to look for
toponyms=PlatLoc

#Combining input text string and list of toponyms
inputs = f"""Text:
{texts}

Toponyms (to detect in the text):
{json.dumps(toponyms)}
"""

# Create response function to recognize and resolve toponyms and detect emotions in 150 character context
# on either side of the toponym.

response = client.responses.create(
  model="gpt-4.1-2025-04-14",
  instructions=instructions,
  input=inputs,
  text={
    "format": {
      "type": "text"
    }
  },
  reasoning={},
  tools=[
    {
      "type": "function",
      "name": "detect_emotions_with_context",
      "description": "Read the input text and list of toponyms provided, look for the toponyms in the text, and detect the most likely emotion on the context surrounding each toponym.",
      "parameters": {
         "type": "object",
         "properties": {
            "found_toponyms": {
               "type": "array",
               "description": "Array of detected emotions in the context around each provided toponym that is found in the text.",
               "items": {
                  "type": "object",
                  "properties": {
                      "toponym": {"type": "string",
                         "description": "The name of the provided toponym that was found in the text."
                      },
                      "emotion": {"type": "string",
                         "enum": ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"],
                         "description": "Emotion detected"
                      },
                      "confidence_score": {"type": "number",
                         "description": "The confidence score for the detected emotion, on a scale of 0 to 1."
                      },
                      "context": {"type": "string",
                         "description": "The block of text (150 chars on either side) around the toponym."
                      }
                  },
                  "required": [
                     "toponym", "emotion", "confidence_score", "context"
                  ],
                  "additionalProperties": False
               }
            }
          },
          "required": ["found_toponyms"],
          "additionalProperties": False
        },
        "strict": True
      }
    ],
  temperature=0,
  max_output_tokens=32000,
  top_p=0.5,
  store=True
)



In [13]:
response.output[0].arguments

'{"found_toponyms":[{"toponym":"Le Chambon-sur-Lignon","emotion":"joy","confidence_score":0.92,"context":"Age 16, November 1941, Le Chambon-sur-Lignon, France. I\'m in Le Chambon-sur-Lignon. It\'s way up on a high plateau in the mountains. It\'s beautiful here. What an amazing difference from Gurs! I\'m staying at La Guespy, which is a home for refugees.  My friend just arrived, and before she came she saw her mother for an hour that morning through the fence, and then they put them all on cattle wagons and they were deported. And now they\'re gone.  We are not sure, but we think they took them to Drancy.  Martin and I are working in the new carpentry shop now. I don\'t really like carpentry, but at least I know how to do it, and at least I am safe."},{"toponym":"Plateau","emotion":"joy","confidence_score":0.80,"context":"I\'m in Le Chambon-sur-Lignon. It\'s way up on a high plateau in the mountains. It\'s beautiful here. What an amazing difference from Gurs! I\'m staying at La Guespy,

In [5]:
#Take response output, convert into json format, then normalize the data that I want 
#and put into a dataframe.

output = json.loads(response.output[0].arguments)
df = pd.json_normalize(output['found_toponyms'], meta=['toponym', 'emotion', 'confidence_score', 'context'])
df

,toponym,emotion,confidence_score,context
0,Chambon-sur-Lignon,joy,0.85,"I received, through, again, Tracy Strong, infl..."
1,Le Chambon-sur-Lignon,joy,0.90,And I was told to get off at a place called Le...
2,Le Chambon,joy,0.85,Later on I found out that he was the cousin of...
3,Le Chambon-sur-Lignon,fear,0.95,"Age 17, February 1943, Le Chambon-sur-Lignon, ..."
4,Le Chambon-sur-Lignon,sadness,0.97,"Age 17, September 1942, Le Chambon-sur-Lignon,..."
5,Le Chambon-sur-Lignon,joy,0.88,"Age 16, November 1941, Le Chambon-sur-Lignon, ..."
6,Le Chambon-sur-Lignon,joy,0.90,"Age 16, February 1942, Le Chambon-sur-Lignon, ..."
7,Le Chambon-sur-Lignon,fear,0.93,"Age 17, September 1942, Le Chambon-sur-Lignon,..."
8,Le Chambon-sur-Lignon,joy,0.87,"Age 17, January 1943, Le Chambon-sur-Lignon, F..."
9,Le Chambon-sur-Lignon,sadness,0.92,"Age 17, March 1943, Le Chambon-sur-Lignon, Fra..."


In [6]:
df.to_csv("Results23B_ToponymsEmotions_smallSubCorpus.csv", encoding="utf-8-sig", index=False, header=True, mode="w+")